In [1]:
import logging

import pickle

import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from torch.distributions import constraints
from torch import nn
import pyro
import pyro.distributions as dist
import pyro.optim as optim
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
from pyro.infer import Predictive
import seaborn as sns
from pyro import poutine
from sklearn import metrics

In [2]:
pyro.set_rng_seed(10)

In [3]:

with open('data_all.pickle', 'rb') as handle:
    data = pickle.load(handle)
print(data.shape)

(1127, 5237)


In [6]:
class PMF_Bayesian(nn.Module):

    #with multivariate gammas that are "somehow?" related through pyro's dependent dimension setting
    #how to define their covariance?
    def __init__(self, train, dim):
        super().__init__()
        """Build the Probabilistic Matrix Factorization model using pymc3.



        """
        self.dim = dim   
        self.data = train.copy()
        self.n, self.m = self.data.shape
        self.map = None
        self.bounds = (0,1)
        self.losses = None
        self.predictions = None


        # Perform mean value imputation
    
        
        # Low precision reflects uncertainty; prevents overfitting.
        # Set to the mean variance across users and items.
        self.alpha_u = (np.mean(self.data, axis=1).mean())**2 / np.std(self.data, axis=1).mean()
        self.alpha_v = (np.mean(self.data, axis=0).mean())**2 / np.std(self.data, axis=0).mean()

        self.beta_u = (np.mean(self.data, axis=1).mean()) / np.std(self.data, axis=1).mean()
        self.beta_v = (np.mean(self.data, axis=0).mean()) / np.std(self.data, axis=0).mean()
       
        self.bias = self.data.mean()


    def model(self,  train):

        drug_plate = pyro.plate("drug_latents", self.n, dim= -1) #independent users
        sideeffect_plate = pyro.plate("sideeffect_latents", self.m, dim= -1) #independent items

        UA = pyro.sample("UA", dist.Gamma(self.alpha_u, self.beta_u).expand([self.dim,self.n]).to_event(2))
            #UA_int = pyro.sample("UAint", dist.Normal(0., 1.))
        
        
        VA = pyro.sample("VA", dist.Gamma(self.alpha_v, self.beta_v).expand([self.dim,self.m]).to_event(2))
            #possibly add intercepts VA_int = pyro.sample("VA", dist.Normal(0., 1.).to_event(1))
       
        u2_plate = pyro.plate("u2_plate", self.n, dim=-2)

        with sideeffect_plate, u2_plate: 
            Y = pyro.sample("target", dist.Poisson(UA.T@VA), obs = train) 
            return Y
            
    def guide(self, train=None):

        d_alpha = pyro.param('d_alpha', torch.ones(self.dim,self.n), constraint=constraints.positive)#*self.user_mean)
        d_beta = pyro.param('d_beta', 0.5*torch.ones(self.dim,self.n), constraint=constraints.positive)
       # int_mean = pyro.param('int_mean', torch.tensor(1.)*self.user_mean)
       # mov_cov = pyro.param('mov_cov', torch.tensor(1.)*0.1,
          #                  constraint=constraints.positive)

        s_alpha = pyro.param('s_alpha', torch.ones(self.dim,self.m), constraint=constraints.positive)#*self.item_mean)
        s_beta = pyro.param('s_beta', 0.5*torch.ones (self.dim,self.m), constraint=constraints.positive)
        drug_plate = pyro.plate("drug_latents", self.n, dim= -1) #independent users
        sideeffect_plate = pyro.plate("sideeffect_latents", self.m, dim= -1) #independent items)

       
        UA = pyro.sample("UA", dist.Gamma(d_alpha, d_beta).to_event(2))
           # UA_int = pyro.sample("UAint", dist.Normal(int_mean, mov_cov).to_event(1))
        
        VA = pyro.sample("VA", dist.Gamma(s_alpha, s_beta).to_event(2))

    
    def train_SVI(self,train, nsteps=250, lr = 0.01, lrd = 1):
        logging.basicConfig(format='%(message)s', level=logging.INFO)
        svi = SVI(self.model,
        self.guide,
        optim.ClippedAdam({"lr": lr, "lrd": lrd}),
        loss=Trace_ELBO())
        losses = []
        for step in range(nsteps):
            elbo = svi.step(torch.from_numpy(train).float())
            losses.append(elbo)
            if step % 10 == 0:
                print("Elbo loss: {}".format(elbo))
        self.losses = losses
        #constrained_params = list(pyro.get_param_store().values())
        #PARAMS = [p.unconstrained() for p in constrained_params]
        #print(PARAMS)
        return losses
    
    def sample_predict(self, nsamples=500 , verbose=True):
        predictive_svi = Predictive(self.model, guide=self.guide, num_samples=nsamples)( None)
        if (verbose):
            for k, v in predictive_svi.items():
                print(f"{k}: {tuple(v.shape)}")
        table = predictive_svi["target"].numpy()
        mc_table = table.mean(axis = 0)
        mc_table_std = table.std(axis = 0)
        mc_table[mc_table < self.bounds[1]] = self.bounds[0]
        mc_table[mc_table >= self.bounds[1]] = self.bounds[1]
        self.predictions = mc_table
        
    
    def rmse(self,test):
        low, high = self.bounds
        test_data = test.copy()
        test_data[test_data < high] = low
        test_data[test_data >= high] = high
        sqerror = abs(test_data - self.predictions) ** 2  # squared error array
        mse = sqerror.sum()/(test_data.shape[0]*test_data.shape[1])
        print("PMF MAP training RMSE: %.5f" % np.sqrt(mse))
        fpr, tpr, thresholds = metrics.roc_curve(test_data.astype(int).flatten(),  self.predictions.astype(int).flatten(), pos_label=1)
        metrics.auc(fpr, tpr)
        print("AUC: %.5f" % metrics.auc(fpr, tpr))
        return np.sqrt(mse) , metrics.auc(fpr, tpr)

    def get_predictions(self):
        return self.predictions


In [11]:
test = PMF_Bayesian(train=data, dim=100)
test.train_SVI(data)



Elbo loss: 15783411.265625
Elbo loss: 15663015.265625
Elbo loss: 16145332.765625
Elbo loss: 15826587.921875
Elbo loss: 15719802.25
Elbo loss: 15592114.84375
Elbo loss: 16002269.5625
Elbo loss: 15610041.640625
Elbo loss: 15560336.8125
Elbo loss: 15778299.484375
Elbo loss: 15540349.015625
Elbo loss: 15444293.75
Elbo loss: 15495942.046875
Elbo loss: 15487900.578125
Elbo loss: 15396425.5
Elbo loss: 15226229.21875
Elbo loss: 15339010.9375
Elbo loss: 15327952.0
Elbo loss: 15188313.953125
Elbo loss: 15354570.0
Elbo loss: 15365459.0625
Elbo loss: 15043918.90625
Elbo loss: 15205022.578125
Elbo loss: 15365884.71875
Elbo loss: 15196678.84375


[15783411.265625,
 15982159.515625,
 15858975.078125,
 15929883.609375,
 16119493.796875,
 15810800.484375,
 16048108.4375,
 15962100.484375,
 15937342.078125,
 15806775.75,
 15663015.265625,
 15990829.375,
 15851723.671875,
 15867087.625,
 15815056.046875,
 15978717.75,
 15919643.703125,
 15788299.546875,
 15660207.359375,
 15762928.828125,
 16145332.765625,
 15920802.90625,
 16016311.21875,
 15984159.96875,
 15796621.125,
 15644638.65625,
 15806779.578125,
 15928478.21875,
 15983178.71875,
 15844046.15625,
 15826587.921875,
 15793722.109375,
 15708210.421875,
 15622728.5625,
 15751240.828125,
 15913168.875,
 15851111.0625,
 15580053.75,
 15758030.65625,
 15696492.1875,
 15719802.25,
 15613640.3125,
 15740376.546875,
 15998692.515625,
 15668583.15625,
 15841526.90625,
 15768233.75,
 15596485.8125,
 15699927.5,
 15769493.234375,
 15592114.84375,
 15720412.875,
 15645728.109375,
 15468635.5,
 15542452.796875,
 15514051.15625,
 15546926.09375,
 15690490.90625,
 15488243.84375,
 15644363.

In [12]:
test.sample_predict(1000)
test.rmse(data)
print(test.get_predictions())
print(data)

UA: (1000, 1, 1, 100, 1127)
VA: (1000, 1, 1, 100, 5237)
target: (1000, 1127, 5237)
PMF MAP training RMSE: 0.36233
AUC: 0.84131
[[1. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 1. 1. 0.]
 ...
 [1. 0. 0. ... 1. 1. 0.]
 [1. 0. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[ 1  0  0 ...  1  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 1  0  0 ...  1  8  0]
 ...
 [ 8  0  0 ... 10 12  0]
 [ 1  0  0 ...  4 25  0]
 [ 0  0  0 ...  0  0  0]]


In [13]:
test.rmse(data)

PMF MAP training RMSE: 0.36233
AUC: 0.84131


(0.36233013243166723, 0.841312632158804)

In [4]:
class PMF_Bayesian2(nn.Module):
    # by default our latent space is 50-dimensional
    # and we use 400 hidden units
    def __init__(self, train, dim):
        super().__init__()
        """Build the Probabilistic Matrix Factorization model using pymc3.



        """
        self.dim = dim   
        self.data = train.copy()
        self.n, self.m = self.data.shape
        self.map = None
        self.bounds = (0,1)
        self.losses = None
        self.predictions = None


        # Perform mean value imputation
    
        
        # Low precision reflects uncertainty; prevents overfitting.
        # Set to the mean variance across users and items.
        self.alpha_u = (np.mean(self.data, axis=1).mean())**2 / np.std(self.data, axis=1).mean()
        self.alpha_v = (np.mean(self.data, axis=0).mean())**2 / np.std(self.data, axis=0).mean()

        self.beta_u = (np.mean(self.data, axis=1).mean()) / np.std(self.data, axis=1).mean()
        self.beta_v = (np.mean(self.data, axis=0).mean()) / np.std(self.data, axis=0).mean()
       
        self.bias = self.data.mean()


    def model(self,train):
            m_10 = torch.zeros(self.n)
            cov_w1 = torch.eye(self.n)
            cov_1 = pyro.sample("cov1", dist.Wishart(covariance_matrix= cov_w1, df = torch.Tensor([2*self.n])))
            mu_1 = pyro.sample("mu_1", dist.MultivariateNormal(m_10, precision_matrix = cov_1))
            m_20 = torch.zeros(self.m)
            cov_w2 = torch.eye(self.m)
            cov_2 = pyro.sample("cov2", dist.Wishart(covariance_matrix= cov_w2, df = torch.Tensor([2*self.m])))
            mu_2 = pyro.sample("mu_2", dist.MultivariateNormal(m_20, precision_matrix = cov_2))
            print(torch.det(cov_1))

            
            drug_plate = pyro.plate("drug_latents", self.n, dim= -1) #independent users
            sideeffect_plate = pyro.plate("sideeffect_latents", self.m, dim= -1) #independent items

            UA = pyro.sample("UA", dist.MultivariateNormal(mu_1, precision_matrix= cov_1))
                #UA_int = pyro.sample("UAint", dist.Normal(0., 1.))
            
            
            VA = pyro.sample("VA", dist.MultivariateNormal(mu_2, precision_matrix =cov_2))
                #possibly add intercepts VA_int = pyro.sample("VA", dist.Normal(0., 1.).to_event(1))
        
            u2_plate = pyro.plate("u2_plate", self.n, dim=-2)

            with sideeffect_plate, u2_plate: 
                Y = pyro.sample("target", dist.Poisson(torch.abs(UA.T@VA)), obs = train) 
                return Y

    def guide( self,train=None, mask=None):

            mu_10= pyro.param('mu_10', torch.zeros(self.n))#*self.user_mean)
            cov_w1 = pyro.param('d_beta', torch.eye(self.n), constraint=constraints.positive_definite)
            mu_20= pyro.param('s_alpha', torch.zeros(self.m))
            cov_w2 = pyro.param('s_beta', torch.eye(self.m), constraint=constraints.positive_definite)
            cov_1 = pyro.sample("cov1", dist.Wishart(covariance_matrix= cov_w1, df = torch.Tensor([2*self.n])))
            cov_2 = pyro.sample("cov2", dist.Wishart(covariance_matrix= cov_w2, df = torch.Tensor([2*self.m])))
            mu_1 = pyro.sample("mu_1", dist.MultivariateNormal(mu_10, precision_matrix = cov_1))
            mu_2 =  pyro.sample("mu_2", dist.MultivariateNormal(mu_20, precision_matrix = cov_2))


            UA =  pyro.sample("UA", dist.MultivariateNormal(mu_1, precision_matrix= cov_1))
            # UA_int = pyro.sample("UAint", dist.Normal(int_mean, mov_cov).to_event(1))
            
            VA = pyro.sample("VA", dist.MultivariateNormal(mu_2, precision_matrix =  cov_2))

    
    def train_SVI(self,train, nsteps=25, lr = 0.01, lrd = 1):
        logging.basicConfig(format='%(message)s', level=logging.INFO)
        svi = SVI(self.model,
        self.guide,
        optim.ClippedAdam({"lr": lr, "lrd": lrd}),
        loss=Trace_ELBO())
        losses = []
        for step in range(nsteps):
            elbo = svi.step(torch.from_numpy(train).float())
            losses.append(elbo)
            if step % 10 == 0:
                print("Elbo loss: {}".format(elbo))
        self.losses = losses
        #constrained_params = list(pyro.get_param_store().values())
        #PARAMS = [p.unconstrained() for p in constrained_params]
        #print(PARAMS)
        return losses
    
    def sample_predict(self, nsamples=500 , verbose=True):
        predictive_svi = Predictive(self.model, guide=self.guide, num_samples=nsamples)( None)
        if (verbose):
            for k, v in predictive_svi.items():
                print(f"{k}: {tuple(v.shape)}")
        table = predictive_svi["target"].numpy()
        mc_table = table.mean(axis = 0)
        mc_table_std = table.std(axis = 0)
        mc_table[mc_table < self.bounds[1]] = self.bounds[0]
        mc_table[mc_table >= self.bounds[1]] = self.bounds[1]
        self.predictions = mc_table
        
    
    def rmse(self,test):
        low, high = self.bounds
        test_data = test.copy()
        test_data[test_data < high] = low
        test_data[test_data >= high] = high
        sqerror = abs(test_data - self.predictions) ** 2  # squared error array
        mse = sqerror.sum()/(test_data.shape[0]*test_data.shape[1])
        print("PMF MAP training RMSE: %.5f" % np.sqrt(mse))
        fpr, tpr, thresholds = metrics.roc_curve(test_data.astype(int).flatten(),  self.predictions.astype(int).flatten(), pos_label=1)
        metrics.auc(fpr, tpr)
        print("AUC: %.5f" % metrics.auc(fpr, tpr))
        return np.sqrt(mse) , metrics.auc(fpr, tpr)

    def get_predictions(self):
        return self.predictions


In [5]:
data2 = data[:,0:2890]
test2 = PMF_Bayesian2(train=data2, dim=100)
test2.train_SVI(data2)
test2.sample_predict(500)
test2.rmse(data2)
print(test2.get_predictions())
print(data2)


/Users/dafnep/Library/Python/3.8/lib/python/site-packages/torch/distributions/wishart.py:231: UserWarning: Singular sample detected.
  warnings.warn("Singular sample detected.")


tensor([inf], grad_fn=<DetLuBasedHelperBackward0>)
Elbo loss: 178107808.0
tensor([inf], grad_fn=<DetLuBasedHelperBackward0>)
tensor([inf], grad_fn=<DetLuBasedHelperBackward0>)
tensor([inf], grad_fn=<DetLuBasedHelperBackward0>)


_LinAlgError: torch.linalg_cholesky: The factorization could not be completed because the input is not positive-definite (the leading minor of order 2228 is not positive-definite).
Trace Shapes:                    
 Param Sites:                    
        mu_10      1127          
       d_beta 1127 1127          
      s_alpha      2890          
       s_beta 2890 2890          
Sample Sites:                    
    cov1 dist    1    | 1127 1127
        value    1    | 1127 1127

In [7]:
#Testing!!!

n,m = data.shape
dim=10
mean_u = 4
mean_v=5
std_v=1
std_u=1
dim=10
alpha_u = 4
alpha_v=5
beta_u=1
beta_v=1
m=3000
drug_code = np.arange(n)
se_code =np.arange(m)
def model():
        m_10 = torch.zeros(n)
        cov_w1 = torch.eye(n)
        cov_1 = pyro.sample("cov1", dist.Wishart(scale_tril= cov_w1, df = torch.Tensor([2*n])))
        mu_1 = pyro.sample("mu_1", dist.MultivariateNormal(m_10, precision_matrix = cov_1))
        m_20 = torch.zeros(m)
        cov_w2 = torch.eye(m)
        cov_2 = pyro.sample("cov2", dist.Wishart(scale_tril= cov_w2, df = torch.Tensor([2*m])))
        mu_2 = pyro.sample("mu_2", dist.MultivariateNormal(m_20, precision_matrix = cov_2))
        print(cov_1)

        
        drug_plate = pyro.plate("drug_latents", n, dim= -1) #independent users
        sideeffect_plate = pyro.plate("sideeffect_latents", m, dim= -1) #independent items

        UA = pyro.sample("UA", dist.MultivariateNormal(mu_1, cov_1))
            #UA_int = pyro.sample("UAint", dist.Normal(0., 1.))
        
        
        VA = pyro.sample("VA", dist.MultivariateNormal(mu_2, cov_2))
            #possibly add intercepts VA_int = pyro.sample("VA", dist.Normal(0., 1.).to_event(1))
       
        u2_plate = pyro.plate("u2_plate", n, dim=-2)

        with sideeffect_plate, u2_plate: 
            Y = pyro.sample("target", dist.Poisson(torch.abs(UA.T@VA))) 
            return Y

def guide( train=None, mask=None):

        mu_1= pyro.param('d_alpha', torch.zeros(n))#*self.user_mean)
        cov_w1 = pyro.param('d_beta', torch.eye(n), constraint=constraints.positive)
       # int_mean = pyro.param('int_mean', torch.tensor(1.)*self.user_mean)
       # mov_cov = pyro.param('mov_cov', torch.tensor(1.)*0.1,
          #                  constraint=constraints.positive)

        mu_2= pyro.param('s_alpha', torch.zeros(m))
        cov_w2 = pyro.param('s_beta', torch.eye(m), constraint=constraints.positive)
        
        cov_1 = pyro.sample("cov1", dist.Wishart(scale_tril= cov_w1, df = torch.Tensor([2*n])))
        cov_2 = pyro.sample("cov2", dist.Wishart(scale_tril= cov_w2, df = torch.Tensor([2*m])))


        UA =  pyro.sample("UA", dist.MultivariateNormal(mu_1, cov_1))
           # UA_int = pyro.sample("UAint", dist.Normal(int_mean, mov_cov).to_event(1))
        
        VA = pyro.sample("VA", dist.MultivariateNormal(mu_2, cov_2))

In [8]:


trace=poutine.trace(guide).get_trace()
trace.compute_log_prob()
print(trace.format_shapes())

trace=poutine.trace(model).get_trace()
trace.compute_log_prob()
print(trace.format_shapes())

Trace Shapes:                    
 Param Sites:                    
      d_alpha      1127          
       d_beta 1127 1127          
      s_alpha      2800          
       s_beta 2800 2800          
Sample Sites:                    
    cov1 dist    1    | 1127 1127
        value    1    | 1127 1127
     log_prob    1    |          
    cov2 dist    1    | 2800 2800
        value    1    | 2800 2800
     log_prob    1    |          
      UA dist    1    | 1127     
        value    1    | 1127     
     log_prob    1    |          
      VA dist    1    | 2800     
        value    1    | 2800     
     log_prob    1    |          
tensor([[[ 2.2811e+03,  1.6974e+01,  1.2097e+01,  ..., -9.2563e+01,
          -8.4243e+00, -5.2117e+01],
         [ 1.6974e+01,  2.2396e+03, -3.9046e+01,  ..., -1.7212e-01,
          -1.8317e+01, -2.2925e+01],
         [ 1.2097e+01, -3.9046e+01,  2.4201e+03,  ...,  7.1174e+01,
           6.4328e+01,  5.0572e+00],
         ...,
         [-9.2563e+01, -1